Import the libraries

In [5]:
import pandas as pd

Import the CSV and head preview

In [6]:
icfes_data_cesar = pd.read_csv("../data/icfes_cols_filtered_no_dupes_BOM.csv")
icfes_data_cesar

,periodo,estu_consecutivo,cole_area_ubicacion,cole_calendario,cole_cod_dane_sede,cole_jornada,cole_naturaleza,cole_nombre_sede,fami_educacionmadre,fami_educacionpadre,desemp_ingles,punt_ingles,punt_matematicas,punt_sociales_ciudadanas,punt_c_naturales,punt_lectura_critica,punt_global
0,20224,SB11202240152772,RURAL,A,320013001752,MAÑANA,OFICIAL,INSTITUCION EDUCATIVA DIVINO NIÑO,Primaria completa,Primaria completa,A-,35.0,38.0,37.0,36.0,36.0,183.0
1,20224,SB11202240042820,RURAL,A,220614000431,MAÑANA,OFICIAL,INSTITUCION EDUCATIVA AGROPECUARIA LOS ANGELES,Primaria completa,Primaria completa,A-,45.0,46.0,47.0,43.0,52.0,234.0
2,20224,SB11202240462114,URBANO,A,120400008286,MAÑANA,OFICIAL,INSTITUCION EDUCATIVA LUIS CARLOS GALAN SARMIENTO,Primaria incompleta,Primaria completa,A1,54.0,37.0,31.0,42.0,41.0,195.0
3,20224,SB11202240377474,RURAL,A,420013001749,MAÑANA,NO OFICIAL,FUNDACION EDUCATIVA NUESTRA SEÑORA DE LAS FLORES,Educación profesional completa,Educación profesional completa,A1,57.0,53.0,61.0,54.0,58.0,283.0
4,20224,SB11202240444419,URBANO,A,120238000136,NOCHE,OFICIAL,INSTITUCION EDUCATIVA INSTITUTO AGRICOLA,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,A-,31.0,33.0,39.0,41.0,35.0,183.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125180,20132,SB11201320354779,URBANO,A,320621001218,MAÑANA,NO OFICIAL,GIMNASIO DE LA PAZ,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,A1,43.0,41.0,NaN,NaN,NaN,NaN
125181,20112,SB11201120442371,RURAL,A,120787007002,MAÑANA,OFICIAL,I.E. ANIBAL MARTINEZ ZULETA,Secundaria (Bachillerato) incompleta,Primaria incompleta,A-,34.0,31.0,NaN,NaN,NaN,NaN
125182,20142,SB11201420287600,URBANO,A,120001068691,TARDE,OFICIAL,IE LEONIDAS ACUÑA,Educación profesional completa,Primaria completa,A1,50.0,50.0,45.0,51.0,38.0,232.0
125183,20122,SB11201220440429,URBANO,A,120011001039,TARDE,OFICIAL,INST. EDU. SAN MIGUEL,Secundaria (Bachillerato) completa,Primaria incompleta,A-,39.0,34.0,NaN,NaN,NaN,NaN


## Cleaning process
### Dealing with duplicates
This was already done, since it was managed during the query process of the data using 'distinct' in the Athena SQL queries. 

### Dealing with NaN values
This was already done, since all values where set on the table schema (strings and doubles). Then, on table properties, we set the property 'use.null.for.invalid.data' as 'true'. Thus, we only have to deal with null values. 

### Dealing with null values
Finally, in order to deal with null values, we are going to check where are they most present, and how many of them are there. 

In [ ]:
df = icfes_data_cesar.copy()

count_null_global_score = df["punt_global"].isnull().sum()
per_null_global_score = (count_null_global_score / len(df)) * 100
print(f"Number of null values in 'punt_global': {count_null_global_score}")
print(f"Percentage of null values in 'punt_global': {per_null_global_score:.2f}%")


Number of null values in 'punt_global': 47610
Percentage of null values in 'punt_global': 38.03%
[20122 20112 20102 20101 20132 20111 20121 20141 20131]


We can see that it's a considerable amount, more than a third of the whole data set. Trough further exploration in Excel, we can see that all the data points that have a null value in *"punt_global"* have null values in *"punt_sociales_ciudadanas"*, *"punt_c_naturales"*, and *"punt_lectura_critica"*. However we are going to make one final check before proceeding. 

In [ ]:
null_by_period = df[df["punt_global"].isnull()].groupby("periodo").size().reset_index(name="null_count")

total_by_period = df.groupby("periodo").size().reset_index(name="total_count")

null_analysis = null_by_period.merge(total_by_period, on="periodo")
null_analysis["null_percentage"] = (null_analysis["null_count"] / null_analysis["total_count"]) * 100

null_analysis = null_analysis.sort_values("null_count", ascending=False)

print("Null values in 'punt_global' by periodo:")
print(null_analysis.to_string(index=False))

Null values in 'punt_global' by periodo:
 periodo  null_count  total_count  null_percentage
   20132       11929        11929            100.0
   20112       11745        11745            100.0
   20122       11576        11576            100.0
   20102       11406        11406            100.0
   20111         241          241            100.0
   20121         235          235            100.0
   20101         179          179            100.0
   20131         168          168            100.0
   20141         131          131            100.0


Since the shown periods have a 100% null percentage on the fields explained below ('puntajes'), and our business questions are mainly of the overall student performance, we consider there is not enough information with this data points, to reflect our student information, thus, we will be removing them completly, and not considering those periods for our results. 

In [13]:
filtered_df = df[df["punt_global"].notnull()]
print(filtered_df)

        periodo  estu_consecutivo cole_area_ubicacion cole_calendario  \
0         20224  SB11202240152772               RURAL               A   
1         20224  SB11202240042820               RURAL               A   
2         20224  SB11202240462114              URBANO               A   
3         20224  SB11202240377474               RURAL               A   
4         20224  SB11202240444419              URBANO               A   
...         ...               ...                 ...             ...   
125174    20142  SB11201420541102               RURAL               A   
125175    20142  SB11201420388714              URBANO               A   
125176    20162  SB11201620145255              URBANO               A   
125182    20142  SB11201420287600              URBANO               A   
125184    20162  SB11201620062546              URBANO               A   

        cole_cod_dane_sede cole_jornada cole_naturaleza  \
0             320013001752       MAÑANA         OFICIAL   
1    

Finally, we can get some insights on the shape, columns, and data type of the df

In [18]:
print(filtered_df.shape)
print("")
print(filtered_df.columns)
print("")
print(filtered_df.describe())

(77575, 17)

Index(['periodo', 'estu_consecutivo', 'cole_area_ubicacion', 'cole_calendario',
       'cole_cod_dane_sede', 'cole_jornada', 'cole_naturaleza',
       'cole_nombre_sede', 'fami_educacionmadre', 'fami_educacionpadre',
       'desemp_ingles', 'punt_ingles', 'punt_matematicas',
       'punt_sociales_ciudadanas', 'punt_c_naturales', 'punt_lectura_critica',
       'punt_global'],
      dtype='str')

            periodo  cole_cod_dane_sede   punt_ingles  punt_matematicas  \
count  77575.000000        7.757500e+04  77508.000000      77575.000000   
mean   20175.791763        1.815180e+11     47.970222         48.634779   
std       27.881222        8.598403e+10     10.962903         11.134026   
min    20142.000000        1.200010e+11      0.000000          0.000000   
25%    20152.000000        1.200018e+11     41.000000         41.000000   
50%    20172.000000        1.204000e+11     47.000000         48.000000   
75%    20194.000000        2.204000e+11     53.000000         56